In [ ]:
!pip install google-api-python-client

# Import libraries
import pandas as pd
import re
from googleapiclient.discovery import build

# Enter unique API key
api_key = input("Please enter your YouTube API key: ")
if not api_key:
    print("Error: API key is required!")
    exit()

# Initialize the YouTube API
youtube = build('youtube', 'v3', developerKey=api_key)

# Your list of video URLs
video_urls = [
    "https://www.youtube.com/watch?v=83IWEkCgFi0",
    "https://www.youtube.com/watch?v=9bdab8_JMrw",
    "https://www.youtube.com/watch?v=AIX8DCZRI18",
    "https://www.youtube.com/watch?v=SoEJijhfHuE",
    "https://www.youtube.com/watch?v=nBYmolDbxgA",
    "https://www.youtube.com/watch?v=IcLjyTCxvRU",
    "https://www.youtube.com/watch?v=Ot1BYJP8Dig",
    "https://www.youtube.com/watch?v=7P1qTwlGeuY",
    "https://www.youtube.com/watch?v=A6wmG7QqaOk"
    "https://www.youtube.com/watch?v=IiZRR1vIlhY"
]

# Function to extract video ID from URL
def extract_video_id(url):
    match = re.search(r"v=([a-zA-Z0-9_-]+)", url)
    return match.group(1) if match else None

# Function to get comments from a video
def get_comments(video_id):
    comments = []
    next_page_token = None

    while True:
        response = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        ).execute()

        for item in response.get("items", []):
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "video_id": video_id,
                "author": comment["authorDisplayName"],
                "published_at": comment["publishedAt"],
                "text": comment["textDisplay"],
                "like_count": comment["likeCount"]
            })

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return comments

# Extract comments for each video
all_comments = []
for url in video_urls:
    video_id = extract_video_id(url)
    print(f"Collecting comments from video: {video_id}")
    try:
        video_comments = get_comments(video_id)
        all_comments.extend(video_comments)
    except Exception as e:
        print(f"Failed for video {video_id}: {e}")

# Convert to DataFrame and save to CSV
df = pd.DataFrame(all_comments)
df.to_csv("youtube_helene_comments.csv", index=False)
print("Comments saved to 'youtube_helene_comments.csv'")